In [51]:
import sys
import pandas as pd
sys.path.append('/home/wangs/rs/lib')
# sys.path.append('/home/wangs/rs/qza')
import os
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import zipfile
import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import imp
import ff

# **数字货币截面因子生成示例**
- 原始数据为 1分钟级K线
- 下面以4h、1day调仓间隔因子为例

## 构造调仓间隔为 4h 的截面因子

In [152]:
# 读取 1分钟级K线数据
pd.read_hdf(f'/home/wangs/data/ba/BTC.h5')

,o,h,l,c,v,ret
0,,,,,,
20190101 000000,3752.22,3775.01,3751.00,3775.00,63.274226,NaN
20190101 000100,3775.00,3776.85,3760.00,3760.00,69.832010,-0.003974
20190101 000200,3759.57,3759.57,3736.98,3749.85,84.820021,-0.002699
20190101 000300,3749.89,3751.31,3743.83,3747.30,33.507780,-0.000680
20190101 000400,3748.49,3752.78,3747.18,3749.30,13.087506,0.000534
...,...,...,...,...,...,...
20240217 231800,50856.50,50856.51,50824.44,50848.00,35.102610,-0.000167
20240217 231900,50847.99,50851.10,50800.00,50800.01,36.052650,-0.000944
20240217 232000,50800.01,50800.01,50760.00,50763.72,40.126460,-0.000714


### 用分钟线构造调仓间隔为 4h 的截面因子
- /mnt/ssd1/wangs/data/ba/ 路径下的分钟数据结构 与ff.read_min()调用的数据式类似，调取目标资产1分钟级K线行情数据；
- 由于存在部分数据中断问题，使用rolling等方法需谨慎；
- 调用示例可参考如下 **4h级** KUR因子构造代码；

In [2]:
def cal_kur_periodly(_df):
    apd, apd1 = _df.v, _df.c.pct_change()

    _mean = apd.mean()
    _std = apd.std()
    logi = apd - _mean > 1 * _std
    logi1 = apd - _mean <= 1 * _std

    _kur_count = len(apd[logi1.shift(1).fillna(False) & logi1.shift(-1).fillna(False) & logi])
    
    return _kur_count

In [3]:
def cal_kur_symly(sym, scale=240): # 4h = 240min
    _data = pd.read_hdf(f'/mnt/ssd1/wangs/data/ba/{sym}.h5')

    time_idx = _data.index.map(lambda x:datetime.datetime.strptime(x,'%Y%m%d %H%M%S'))
    _data.index = time_idx
    
    _group_K_x_min = _data.resample(f'{str(scale)}min',closed='left',label='right')
    
    # 遍历每个持仓周期
    p_dct = {}
    for _t, _df in _group_K_x_min:
        p_dct[str(_t)] = cal_kur_periodly(_df)
    
    return p_dct

In [ ]:
ff.read_biance

In [4]:
# 品种名称（带.h5）
coins_name = os.listdir('/mnt/ssd1/wangs/data/ba/')

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/ssd1/wangs/data/ba/'

In [148]:
KURS_dct = {}
for _sym in tqdm(coins_name):
    KURS_dct[_sym[:-3]] = cal_kur_symly(_sym[:-3])

100%|███████████████████████████████████████████████████████████████████████████████████| 28/28 [08:34<00:00, 18.36s/it]


In [150]:
c_KURS_df = pd.DataFrame(KURS_dct).sort_index().T.shift(1,axis=1)
# 注意日期格式保持一致
c_KURS_df.columns = [col.replace('-','') for col in c_KURS_df.columns]

In [164]:
c_KURS_df.head(10)

,20190101 04:00:00,20190101 08:00:00,20190101 12:00:00,20190101 16:00:00,20190101 20:00:00,20190102 00:00:00,20190102 04:00:00,20190102 08:00:00,20190102 12:00:00,20190102 16:00:00,...,20240216 12:00:00,20240216 16:00:00,20240216 20:00:00,20240217 00:00:00,20240217 04:00:00,20240217 08:00:00,20240217 12:00:00,20240217 16:00:00,20240217 20:00:00,20240218 00:00:00
1000SATS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,13.0,12.0,3.0,12.0,11.0,10.0,22.0,5.0,12.0
ADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.0,14.0,17.0,11.0,7.0,15.0,12.0,14.0,7.0,13.0
APE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.0,16.0,10.0,9.0,19.0,10.0,11.0,8.0,7.0,3.0
ARB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.0,13.0,12.0,16.0,7.0,16.0,15.0,16.0,15.0,13.0
ARPA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,19.0,9.0,9.0,16.0,11.0,14.0,6.0,9.0,17.0
ATOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,15.0,20.0,13.0,7.0,17.0,16.0,14.0,9.0,8.0
AVAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,14.0,10.0,3.0,8.0,13.0,17.0,13.0,15.0,16.0
BAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.0,13.0,10.0,9.0,7.0,11.0,13.0,9.0,12.0,14.0
BNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,5.0,15.0,10.0,14.0,8.0,10.0,9.0,8.0,20.0
BTC,NaN,2.0,17.0,22.0,11.0,14.0,7.0,2.0,3.0,13.0,...,12.0,6.0,7.0,8.0,9.0,14.0,7.0,15.0,15.0,11.0


## 构造调仓间隔为 1day 的截面因子

### 用分钟线构造调仓间隔为 1day 的截面因子
- /mnt/ssd1/wangs/data/ba/ 路径下的分钟数据结构 与ff.read_min()调用的数据式类似，调取目标资产1分钟级K线行情数据；
- 由于存在部分数据中断问题，使用rolling等方法需谨慎；
- 调用示例可参考如下 **1day级** KUR因子构造代码；

In [13]:
def cal_kur_periodly_1d(_df):
    apd, apd1 = _df.v, _df.c.pct_change()

    _mean = apd.mean()
    _std = apd.std()
    logi = apd - _mean > 1 * _std
    logi1 = apd - _mean <= 1 * _std

    _kur_count = len(apd[logi1.shift(1).fillna(False) & logi1.shift(-1).fillna(False) & logi])
    
    return _kur_count

In [27]:
def cal_kur_symly_1d(scale=1440): # 1day = 1440min
    _data = ff.read_binance('BTC')

    time_idx = _data.index.map(lambda x:datetime.datetime.strptime(x,'%Y%m%d %H%M%S'))
    _data.index = time_idx
    
    _group_K_x_min = _data.resample(f'{str(scale)}min',closed='left',label='right')
    
    # 遍历每个持仓周期
    p_dct = {}
    for _t, _df in _group_K_x_min:
        p_dct[str(_t)] = cal_kur_periodly_1d(_df)
    
    return p_dct

**除了 cal_kur_symly() 函数的 scale 参数以外，其余都与 4h 级别因子一致**

In [160]:
KURS_dct_1d = {}
for _sym in tqdm(coins_name):
    KURS_dct_1d[_sym[:-3]] = cal_kur_symly_1d(_sym[:-3])

100%|███████████████████████████████████████████████████████████████████████████████████| 28/28 [05:30<00:00, 11.81s/it]


In [161]:
c_KURS_1d_df = pd.DataFrame(KURS_dct_1d).sort_index().T.shift(1,axis=1)
# 注意日期格式保持一致
c_KURS_1d_df.columns = [col.replace('-','') for col in c_KURS_1d_df.columns]

In [165]:
c_KURS_1d_df.head(10)

,20190102 00:00:00,20190103 00:00:00,20190104 00:00:00,20190105 00:00:00,20190106 00:00:00,20190107 00:00:00,20190108 00:00:00,20190109 00:00:00,20190110 00:00:00,20190111 00:00:00,...,20240209 00:00:00,20240210 00:00:00,20240211 00:00:00,20240212 00:00:00,20240213 00:00:00,20240214 00:00:00,20240215 00:00:00,20240216 00:00:00,20240217 00:00:00,20240218 00:00:00
1000SATS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,60.0,50.0,37.0,57.0,24.0,26.0,14.0,37.0,76.0,28.0
ADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,19.0,61.0,67.0,95.0,73.0,57.0,61.0,54.0,50.0,59.0
APE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,71.0,9.0,50.0,40.0,51.0,34.0,51.0,69.0,72.0
ARB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,64.0,38.0,37.0,75.0,58.0,36.0,38.0,69.0,56.0,61.0
ARPA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,11.0,12.0,57.0,50.0,22.0,57.0,25.0,31.0,40.0
ATOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.0,42.0,46.0,50.0,61.0,17.0,53.0,69.0,51.0,77.0
AVAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96.0,44.0,33.0,72.0,33.0,34.0,45.0,57.0,62.0,43.0
BAKE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,18.0,56.0,71.0,71.0,48.0,70.0,31.0,52.0,40.0,72.0
BNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,67.0,31.0,66.0,70.0,79.0,72.0,29.0,65.0,26.0,37.0
BTC,NaN,34.0,46.0,50.0,39.0,69.0,54.0,26.0,38.0,30.0,...,83.0,43.0,29.0,31.0,27.0,19.0,31.0,32.0,38.0,58.0


In [21]:
data = ff.read_binance('BTC')
time_idx = data.index.map(lambda x:datetime.datetime.strptime(x,'%Y%m%d %H%M%S'))
data.index = time_idx

In [28]:
cal_kur_symly_1d()

{'2019-01-02 00:00:00': 34,
 '2019-01-03 00:00:00': 46,
 '2019-01-04 00:00:00': 50,
 '2019-01-05 00:00:00': 39,
 '2019-01-06 00:00:00': 69,
 '2019-01-07 00:00:00': 54,
 '2019-01-08 00:00:00': 26,
 '2019-01-09 00:00:00': 38,
 '2019-01-10 00:00:00': 30,
 '2019-01-11 00:00:00': 21,
 '2019-01-12 00:00:00': 22,
 '2019-01-13 00:00:00': 17,
 '2019-01-14 00:00:00': 46,
 '2019-01-15 00:00:00': 19,
 '2019-01-16 00:00:00': 35,
 '2019-01-17 00:00:00': 32,
 '2019-01-18 00:00:00': 43,
 '2019-01-19 00:00:00': 54,
 '2019-01-20 00:00:00': 15,
 '2019-01-21 00:00:00': 19,
 '2019-01-22 00:00:00': 52,
 '2019-01-23 00:00:00': 11,
 '2019-01-24 00:00:00': 52,
 '2019-01-25 00:00:00': 39,
 '2019-01-26 00:00:00': 49,
 '2019-01-27 00:00:00': 27,
 '2019-01-28 00:00:00': 18,
 '2019-01-29 00:00:00': 44,
 '2019-01-30 00:00:00': 64,
 '2019-01-31 00:00:00': 53,
 '2019-02-01 00:00:00': 39,
 '2019-02-02 00:00:00': 61,
 '2019-02-03 00:00:00': 74,
 '2019-02-04 00:00:00': 37,
 '2019-02-05 00:00:00': 58,
 '2019-02-06 00:00:0

In [52]:
# 所有加密货币的名字
currency_names = ['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'EOS', 'BNB', 'XLM', 'TRX', 'ADA']

# 读取一个币种的高开低收成交量
coin = "BTC"
ff.read_binance(coin)
# 预处理要用到的工具函数
freq = 15 # 15分钟为周期采样
resample_rule = "{}min".format(freq)

# Convert index from str into pd.Datetime
def timing(df):
    df.index = pd.to_datetime(df.index)
    df.index.name = None
    return df

def naming(df, name):
    df.name = name
    return df

def phase_transformer(s): # s = "20220101 010300"
    minute = int(s[11:13])
    return f"{s[:11]}{minute-minute%freq:02d}"

start, end = "20200101", "20240101"
# 读取数据
def currency_data(currency_name):
    global start, end
    data = ff.read_binance(currency_name).rename(columns={'o':'open', 'h':'high', 'l':'low', 'c':'close', 'v':'volume'}).loc[start:end]
    data['tradedate'] = data.index.str[:8]
    data['tradehour'] = data.index.str[:11]
    data['tradephase'] = data.index.str[:15]
    data['tradephase'] = data.tradephase.apply(phase_transformer) # 如果要重采样的话，使用函数phase_transformer进行时间切分
    data = timing(data) # 将data的行索引转化为pd.datetime形式
    return data.reindex(pd.date_range(start=pd.to_datetime(start), 
                                      end=pd.to_datetime(end), 
                                      freq="1min", inclusive="left")) # 补充上缺失的时间索引
data_btc = currency_data('BTC')

In [95]:
btc_ret = data_btc['close']/data_btc['close'].shift(1)
btc_ret_std = btc_ret.rolling(window = 240,closed = 'right').std()
filter_index = ['00:00:00','04:00:00','08:00:00','12:00:00','16:00:00','20:00:00']
btc_ret_std_4 = btc_ret_std[(btc_ret_std).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
btc_ret_std_mean_4 = btc_ret_std_4.rolling(window = 20,closed = 'right').mean()
btc_ret_std_std_4 = btc_ret_std_4.rolling(window = 20,closed = 'right').std()
UID_4 = btc_ret_std_std_4/btc_ret_std_mean_4
UID_4

2020-01-01 00:00:00         NaN
2020-01-01 04:00:00         NaN
2020-01-01 08:00:00         NaN
2020-01-01 12:00:00         NaN
2020-01-01 16:00:00         NaN
                         ...   
2023-12-31 04:00:00    0.233933
2023-12-31 08:00:00    0.240006
2023-12-31 12:00:00    0.247548
2023-12-31 16:00:00    0.259330
2023-12-31 20:00:00    0.261723
Name: close, Length: 8766, dtype: float64

In [96]:
btc_ret = data_btc['close']/data_btc['close'].shift(1)
btc_ret_std = btc_ret.rolling(window = 1440,closed = 'right').std()
filter_index = ['00:00:00']
btc_ret_std_24 = btc_ret_std[(btc_ret_std).index.strftime("%Y-%m-%d %H:%M:%S").str.endswith(tuple(filter_index))]
btc_ret_std_mean_24 = btc_ret_std_24.rolling(window = 20,closed = 'right').mean()
btc_ret_std_std_24 = btc_ret_std_24.rolling(window = 20,closed = 'right').std()
UID_24 = btc_ret_std_std_24/btc_ret_std_mean_24
UID_24

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04         NaN
2020-01-05         NaN
                ...   
2023-12-27    0.313823
2023-12-28    0.314522
2023-12-29    0.312306
2023-12-30    0.306718
2023-12-31    0.280039
Name: close, Length: 1461, dtype: float64